In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import compute_sample_weight
import seaborn as sns

In [74]:
train = pd.read_csv('../titanic/train.csv')
test = pd.read_csv('../titanic/test.csv')
train['dataset'] = 'train'
test['dataset'] = 'test'
test['Survived'] = np.nan
data = pd.concat([train, test])
data.head()

/home/tien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,dataset
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,train
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,train
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,train
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,train
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,train


In [109]:
def name1(name):
    return name.split(", ")[0]
data['name1'] = data.Name.apply(lambda x: name1(x))
data['name1_len'] = data.name1.apply(lambda x: len(x))

def name_type(name):
    name_temp = name.split(", ")[1]
    return name_temp.split(". ")[0]
data['name_type'] = data.Name.apply(lambda x: name_type(x))
data['name_type_len'] = data.name_type.apply(lambda x: len(x))

def name2(name):
    name_temp = name.split(", ")[1]
    name_temp1 = name_temp.split('. ')[1]
    return name_temp1.split(' (')[0]
data['name2'] = data['Name'].apply(lambda x: name2(x))

def name2_len(name2):
    return len(name2.split(" "))
data['name2_len'] = data['name2'].apply(lambda x: name2_len(x))

def married(name):
    name_len = len(name.split(' ('))
    if name_len==1:
        return False
    else:
        return True
data['married'] = data['Name'].apply(lambda x: married(x))

def real_name(name):
    name_temp = name.split(", ")[1]
    name_temp1 = name_temp.split('. ')[1]
    return name_temp1.split(' (')[-1].replace(")", "")
data['real_name'] = data['Name'].apply(lambda x: real_name(x))

def real_name_len(real_name):
    return len(real_name.split(" "))
data['real_name_len'] = data['real_name'].apply(lambda x: real_name_len(x))

def ticket_no(ticket):
    try:
        value = int(ticket.split(" ")[-1])
    except:
        value = 0
    return value
data['ticket_no'] = data.Ticket.apply(lambda x: ticket_no(x))

def ticket_type(ticket):
    if len(ticket.split(" "))==1:
        try:
            remove_part = str(int(ticket))
        except:
            remove_part = ''
        typ = ticket.replace(remove_part, '')
    else:
         typ = ticket.replace(ticket.split(" ")[-1], '')
    return typ
data['ticket_type'] = data.Ticket.apply(lambda x: ticket_type(x))

data.Age.fillna('noAge', inplace=True)
data.Age = train.Age.astype('category')
data['Cabin'].fillna('nocabin', inplace=True)
data.Fare.fillna(train.Fare.mean(), inplace=True)

def create_lookup_table(df, feat):
    return df.groupby(feat).Survived.agg(['count', 'mean'])

def asign_value(table, feat, typ):
    try:
        value = table.loc[feat, typ]
    except:
        value = 0
    return value

In [122]:
float_feats  =['Fare']

cat_feats = ['name_type', 'Pclass', 'Embarked', 'Sex', 'name_type_len', 'Age', 'ticket_type']
for feat in cat_feats:
    data[feat] = data[feat].astype('category')
    data[feat] = data[feat].cat.codes
    
int_feats = ['Parch', 'SibSp']
for feat in int_feats:
    data[feat] = data[feat].astype('int64')
    
bool_feats = ['married']
for feat in bool_feats:
    data[feat] = data[feat].astype('bool')

In [123]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.model_selection import train_test_split

train = data[data.dataset=='train']
test = data[data.dataset=='test']

chosen_feats = float_feats + cat_feats + bool_feats + int_feats

param = {
    'bagging_freq': 40,          'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 0.65,     'learning_rate': 0.1,
    'max_depth': -1,             'metric':'binary_error', 'min_data_in_leaf': 3,   
    'min_sum_hessian_in_leaf': 5.0,   'num_leaves': 15,            
    'num_threads': 8,              'tree_learner': 'serial',   'objective': 'binary',
    'reg_alpha': 0.6002650970728192, 'reg_lambda': 0.6003427518866501,'verbosity': 1,
    "seed": 99999
}

folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=99999)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
labels = train.Survived
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train.Age.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][chosen_feats], label=labels.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][chosen_feats], label=labels.iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000, valid_sets = [trn_data, val_data], verbose_eval=50, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][chosen_feats], num_iteration=clf.best_iteration)
    predictions += clf.predict(test[chosen_feats], num_iteration=clf.best_iteration) / folds.n_splits

/home/tien/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Fold 0
Training until validation scores don't improve for 200 rounds
[50]	training's binary_error: 0.124579	valid_1's binary_error: 0.171717
[100]	training's binary_error: 0.117845	valid_1's binary_error: 0.164983
[150]	training's binary_error: 0.0993266	valid_1's binary_error: 0.164983
[200]	training's binary_error: 0.0959596	valid_1's binary_error: 0.158249
[250]	training's binary_error: 0.0942761	valid_1's binary_error: 0.158249
[300]	training's binary_error: 0.0942761	valid_1's binary_error: 0.158249
[350]	training's binary_error: 0.0875421	valid_1's binary_error: 0.158249
[400]	training's binary_error: 0.0808081	valid_1's binary_error: 0.161616
[450]	training's binary_error: 0.0841751	valid_1's binary_error: 0.161616
[500]	training's binary_error: 0.0808081	valid_1's binary_error: 0.164983
[550]	training's binary_error: 0.0791246	valid_1's binary_error: 0.164983
Early stopping, best iteration is:
[366]	training's binary_error: 0.0824916	valid_1's binary_error: 0.154882
Fold 1
Trai

In [124]:
from math import sqrt
import statistics

def binary_error(train, pred):
    pred = [1 if i>0.5 else 0 for i in pred]
    return (train == pred).sum()/len(oof)
    
print("acc: {:<8.5f}".format(binary_error(train.Survived, oof)))
feature_imp = pd.DataFrame({'Value':clf.feature_importance(),'Feature':chosen_feats})
data1=feature_imp.sort_values(by="Value", ascending=False)
data1

acc: 0.84624 


,Value,Feature
0,577,Fare
6,404,Age
7,126,ticket_type
3,71,Embarked
2,64,Pclass
9,52,Parch
10,44,SibSp
1,43,name_type
4,41,Sex
5,40,name_type_len


In [125]:
sub = pd.DataFrame({"PassengerId": test.PassengerId.values})
sub["Survived"] = [1 if i>0.5 else 0 for i in predictions]
sub['PassengerId'] = sub['PassengerId'].astype('int64')
sub.to_csv("submission_lightgbm.csv", index=False, header=True)